<a href="https://colab.research.google.com/github/riotamoriya/kaggle/blob/main/tablar_nov_2021%5Banalysis%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title Set the environment
%matplotlib inline

import seaborn as sns
import sys
import csv
import datetime
import operator
import joblib
import warnings
from pathlib import Path


warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from collections import OrderedDict
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from scipy.stats import norm, skew, probplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
!pip install category_encoders
from category_encoders.target_encoder import TargetEncoder
from sklearn.model_selection import StratifiedKFold

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 20

     |████████████████████████████████| 82 kB 350 kB/s 


In [3]:
if 'google.colab' in sys.modules:  # colab環境
    INPUT = Path('/content/input/')
elif 'kaggle_web_client' in sys.modules:  # kaggle環境
    INPUT = Path('../input/')


from requests import get
name_notebook = get('http://172.28.0.2:9000/api/sessions').json()[0]['name']
print(name_notebook)


from google.colab import drive
drive.mount('/content/drive')

!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/Kaggle_emwa/kaggle.json ~/.kaggle
!pip install kaggle
!pip install --upgrade --force-reinstall --no-deps kaggle
# !kaggle competitions list

tablar-nov-2021%5Banalysis%5D.ipynb
Mounted at /content/drive
     |████████████████████████████████| 58 kB 3.2 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=45fcbbe4a1b7ee7811dd38756d31913ea719973f5dce42280942fd61c3bc8aec
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [4]:
tryproblem = 'tabular-playground-series-nov-2021'
tryproblem += '.zip'

!kaggle competitions download -c tabular-playground-series-nov-2021
import zipfile
with zipfile.ZipFile(tryproblem) as existing_zip:
    existing_zip.extractall('.')

df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')

 97% 417M/428M [00:04<00:00, 90.3MB/s]
100% 428M/428M [00:04<00:00, 101MB/s] 


In [32]:
from sklearn.model_selection import train_test_split

random_state_val =42
test_size_val =0.2
train,validation = train_test_split(df_train, test_size = test_size_val, random_state = random_state_val)

In [33]:
y_nm = 'target'

df_train_x = train.drop(y_nm, axis = 1)
df_train_y = pd.DataFrame(train[y_nm])

df_val_x = validation.drop(y_nm, axis = 1)
df_val_y = pd.DataFrame(validation[y_nm])

df_test_x = df_test

In [34]:
num_cols = [col for col in df_train_x.columns if df_train_x[col].dtype in ["float16","float32","float64"]]
cat_cols = [col for col in df_train_x.columns if df_train_x[col].dtype not in ["float16","float32","float64"]]

In [35]:
y = train["target"].copy()

for cols in cat_cols:
    enc = TargetEncoder(cols=[cols])
    df_train_x = enc.fit_transform(df_train_x, y)
    df_val_x = enc.transform(df_val_x)
    df_test_x = enc.transform(df_test_x)


In [36]:
scaler = QuantileTransformer()
scaler.fit(df_train_x)
df_train_x = pd.DataFrame(scaler.transform(df_train_x))
df_val_x = pd.DataFrame(scaler.transform(df_val_x))
df_test_x = pd.DataFrame(scaler.transform(df_test_x))

In [37]:
XGBClassifier = xgb.XGBClassifier(max_depth = 8,
                                 learning_rate = 0.005,
                                 n_estimators = 10000,
                                 objective = 'binary:logistic',
                                 tree_method = 'gpu_hist',
                                 booster = 'gbtree',
                                 gamma = 0.64,
                                 max_delta_step = 3,
                                 min_child_weight = 7,
                                 subsample = 0.7,
                                 colsample_bytree = 0.8,
                                 n_jobs = -1
                                 )

In [38]:
start = datetime.datetime.now()
xgb = XGBClassifier.fit(df_train_x.values,
                       df_train_y.values.ravel(),
                       eval_set = [(df_train_x.values, df_train_y), (df_val_x.values, df_val_y)], 
                       eval_metric = 'auc',
                       early_stopping_rounds = 15,
                       verbose = True)
end = datetime.datetime.now()
end-start

[0]	validation_0-auc:0.652844	validation_1-auc:0.637278
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 15 rounds.
[1]	validation_0-auc:0.671792	validation_1-auc:0.656454
[2]	validation_0-auc:0.681587	validation_1-auc:0.665041
[3]	validation_0-auc:0.685298	validation_1-auc:0.668716
[4]	validation_0-auc:0.688399	validation_1-auc:0.671313
[5]	validation_0-auc:0.689492	validation_1-auc:0.672253
[6]	validation_0-auc:0.689599	validation_1-auc:0.672333
[7]	validation_0-auc:0.689867	validation_1-auc:0.672334
[8]	validation_0-auc:0.689833	validation_1-auc:0.67246
[9]	validation_0-auc:0.691145	validation_1-auc:0.673539
[10]	validation_0-auc:0.691549	validation_1-auc:0.673758
[11]	validation_0-auc:0.693193	validation_1-auc:0.675355
[12]	validation_0-auc:0.693087	validation_1-auc:0.675203
[13]	validation_0-auc:0.693326	validation_1-auc:0.675179
[14]	validation_0-auc:0.694846	validation_1-auc:0.676459

datetime.timedelta(seconds=2971, microseconds=308548)